In [2]:
import chromadb as cdb

In [3]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 13293816
      0 drwxr-xr-x   13 kariato  staff   416B Feb 11 14:17 ./
      0 drwxr-xr-x    7 kariato  staff   224B Feb 10 21:29 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Feb 10 14:58 .DS_Store
    464 -rw-r--r--@   1 kariato  staff   232K Feb 10 16:00 9deec473-6ab3-4c90-a2ec-86ae93b0a6e6.pdf
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
2785400 -rw-rw-r--@   1 kariato  staff   1.3G Sep 19  2019 emails.csv
2109312 -rw-r--r--    1 kariato  staff   1.0G Feb 10 14:24 enron.sql
 427936 -rw-r--r--@   1 kariato  staff   209M Feb 10 14:24 enron.sql.gz
 865736 -rw-r--r--@   1 kariato  staff   423M Feb 10 14:35 enron_mail_20150507.tar.gz
6363112 -rw-r--r--    1 kariato  staff   3.0G Feb 11 14:17 enron_processed.csv
      0 drwxr-xr-x@  11 kariato  staff   352B Nov 10  2004 enron_with_categories/
   8840 -rw-r--r--@   1 kariato  staff   4.3M Feb 10 14:25 enron_with_categories.tar.gz
      0 drwxr-xr-x@ 152 kari

In [5]:
dir(cdb)

['API',
 'Client',
 'ClientStartEvent',
 'Collection',
 'CollectionMetadata',
 'Dict',
 'Documents',
 'EmbeddingFunction',
 'Embeddings',
 'EphemeralClient',
 'GetResult',
 'HttpClient',
 'IDs',
 'IN_COLAB',
 'Include',
 'Metadata',
 'PersistentClient',
 'QueryResult',
 'Settings',
 'System',
 'Telemetry',
 'UpdateCollectionMetadata',
 'Where',
 'WhereDocument',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__settings',
 '__spec__',
 '__version__',
 'api',
 'chromadb',
 'config',
 'configure',
 'errors',
 'get_settings',
 'is_client',
 'logger',
 'logging',
 'sqlite3',
 'telemetry',
 'types',
 'utils']